In [6]:
import pandas as pd
import seaborn as sns


In [20]:
df_path_interim = "../data/interim/"
df_player_rating_stats = pd.read_csv(df_path_interim + "df_player_rating_stats.csv")

In [23]:
df_player_rating_stats.describe()

,player_age,player_id,league_id,team_id,gp,gs,min,fgm,fga,fg_pct,...,rating,ptspg,rebpg,orebpg,drebpg,astpg,stlpg,blkpg,tovpg,pfpg
count,2872.000000,2.872000e+03,2872.0,2.872000e+03,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2858.000000,...,2278.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000
mean,25.179666,8.813979e+05,0.0,1.748569e+09,51.751045,30.127437,1329.075905,229.868733,495.672006,0.455852,...,76.739684,10.699563,4.288728,1.017304,3.271424,2.336385,0.775345,0.487464,1.362369,1.895386
std,3.712560,8.165422e+05,0.0,4.704296e+08,23.611648,28.852508,848.869392,180.946767,382.640320,0.092441,...,6.808460,6.617193,2.639204,0.854483,1.953895,2.070820,0.462799,0.476811,0.897920,0.757796
min,19.000000,2.544000e+03,0.0,1.610613e+09,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,49.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,2.023570e+05,0.0,1.610613e+09,33.000000,2.000000,590.750000,80.000000,178.000000,0.413927,...,72.000000,5.666667,2.420789,0.426557,1.889849,0.928571,0.444444,0.173269,0.714286,1.409965
50%,25.000000,2.040280e+05,0.0,1.610613e+09,58.000000,21.000000,1298.500000,196.500000,430.000000,0.448591,...,76.000000,9.398113,3.800000,0.751712,2.947332,1.666667,0.712329,0.347826,1.153846,1.921311
75%,27.000000,1.628464e+06,0.0,1.610613e+09,72.000000,58.000000,2024.250000,338.000000,729.000000,0.495033,...,80.000000,14.646175,5.507777,1.340685,4.231004,3.104674,1.028675,0.633654,1.814815,2.427143
max,41.000000,4.888947e+06,0.0,4.831838e+09,82.000000,82.000000,3388.000000,875.000000,1941.000000,1.000000,...,99.000000,36.128205,15.987179,5.432099,11.540541,11.738462,2.769231,3.684932,5.728395,4.875000


In [32]:
df_player_rating_stats.columns

Index(['full_name', 'season_id', 'player_age', 'player_id', 'league_id',
       'team_id', 'gp', 'gs', 'min', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a',
       'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl',
       'blk', 'tov', 'pf', 'pts', 'id', 'rating', 'ptspg', 'rebpg', 'orebpg',
       'drebpg', 'astpg', 'stlpg', 'blkpg', 'tovpg', 'pfpg'],
      dtype='object')

In [80]:
df_model = df_player_rating_stats.drop(['league_id', 'team_id', 'min', 'id'], axis = 1)

In [81]:
df_model = df_model.drop(['full_name', 'season_id', 'player_id', 'fg_pct', 'fg3_pct', 'ft_pct'], axis = 1)

In [42]:
from scipy import stats
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


In [91]:
def score(X, y, model = RandomForestRegressor()):
    
    score = cross_val_score(
        model, X, y, cv = 5, scoring = "neg_mean_absolute_error",
    )
    
    return - score.mean()

In [92]:
df_model_train = df_model.dropna(subset = ['rating'])
df_model_test = df_model[df_model['rating'].isnull()]

In [93]:
df_model_train.dtypes

player_age    float64
gp              int64
gs              int64
fgm             int64
fga             int64
fg3m            int64
fg3a            int64
ftm             int64
fta             int64
oreb            int64
dreb            int64
reb             int64
ast             int64
stl             int64
blk             int64
tov             int64
pf              int64
pts             int64
rating        float64
ptspg         float64
rebpg         float64
orebpg        float64
drebpg        float64
astpg         float64
stlpg         float64
blkpg         float64
tovpg         float64
pfpg          float64
dtype: object

In [94]:
X = df_model_train.copy()
y = X.pop('rating')

baseline_score = score(X, y)
print(f"Baseline score: {baseline_score:.5f} neg_root_mean_squared_error")

Baseline score: 2.75454 neg_root_mean_squared_error
